# Chapter 3

In [47]:
# Add these here as they are required later

using BenchmarkTools, LinearAlgebra, StaticArrays

---

### Solve Basel series _(again)_

In [ ]:
# Reciprocal square

function rsq(x)
  y = 1/(x*x)
  return y
end


In [45]:
# Test the rsq() function out

map([1,2,3,4]) do x
  rsq(x)
end later

4-element Vector{Float64}:
 1.0
 0.25
 0.1111111111111111
 0.0625

In [46]:
# Sum up first million (10^6) term

sum(collect(1:10^6)) do x
  rsq(x)
end

1.6449330668487272

---

### Collatz conjecture 

The Hailstone sequence of numbers can be generated from a starting positive integer, n by:

- If n is 1 then the sequence ends.
- If n is even then the next n of the sequence = n/2
- If n is odd then the next n of the sequence = (3 * n) + 1

The Collatz conjecture is that the hailstone sequence for any starting number always terminates.

In [3]:
function hailstone(n)
   k = 1
   a = [n]
   while n > 1 
      n = (n % 2 == 0) ? n >> 1 : 3n + 1 
      push!(a,n)
      k += 1
   end
   a
end

hailstone (generic function with 1 method)

In [4]:
hailstone(17)

13-element Vector{Int64}:
 17
 52
 26
 13
 40
 20
 10
  5
 16
  8
  4
  2
  1

In [5]:
(m,s) = hailstone(1000000)

153-element Vector{Int64}:
 1000000
  500000
  250000
  125000
   62500
   31250
   15625
   46876
   23438
   11719
   35158
   17579
   52738
       ⋮
      53
     160
      80
      40
      20
      10
       5
      16
       8
       4
       2
       1

---

---

### Pattern Matching

In [7]:
using Match

In [8]:
allodds(x) = @match x begin
    m::Int, if iseven(m) || (m < 3) end        => "Not a valid choice"
    n::Int, if n == 3 || n == 5 || n == 7 end  => "$x is prime"
    _  =>  "By induction all numbers are prime"
end

allodds (generic function with 1 method)

In [9]:
allodds(3)

"3 is prime"

In [10]:
allodds(6)

"Not a valid choice"

In [11]:
allodds(9)

"By induction all numbers are prime"

### Solve Queens problem

In [12]:
struct Queen
    x::Int
    y::Int
end

In [13]:
qhorz(qa, qb) = qa.x == qb.x;
qvert(qa, qb) = qa.y == qb.y;
qdiag(qa, qb) = abs(qa.x - qb.x) == abs(qa.y - qb.y);

qhvd(qa, qb) = qhorz(qa, qb) || qvert(qa, qb) || qdiag(qa, qb);
qany(testq, qs) = any(q -> qhvd(testq, q), qs);

In [14]:
function qsolve(nsqsx, nsqsy, nqs, presqs = ())
    nqs == 0 && return presqs
    for xsq in 1:nsqsx
        for ysq in 1:nsqsy
            testq = Queen(xsq, ysq)
            if !qany(testq, presqs)
                tryqs = (presqs..., testq)
                maybe = qsolve(nsqsx, nsqsy, nqs - 1, tryqs)
                maybe !== nothing && return maybe
            end
        end
    end
    return nothing
end

qsolve(nqs) = qsolve(nqs, nqs, nqs)

qsolve (generic function with 3 methods)

In [15]:
qsolve(8)

(Queen(1, 1), Queen(2, 5), Queen(3, 8), Queen(4, 6), Queen(5, 3), Queen(6, 7), Queen(7, 2), Queen(8, 4))

---

## Persistence

What is so special about 277777788888899?


In [ ]:
function persistence(n::Integer)
  @assert n > 0
  s = string(n)
  j = 0
  while length(s) > 1
    j += 1
    s = string(prod(parse.(Int64,split(s,""))))
    @show s
  end
  return j
end

In [2]:
m = persistence(277777788888899);
println("Terminates in $m steps")

s = "4996238671872"
s = "438939648"
s = "4478976"
s = "338688"
s = "27648"
s = "2688"
s = "768"
s = "336"
s = "54"
s = "20"
s = "0"
Terminates in 11 steps


---

## A Vehicle module

In [16]:
module Vehicles

export Contact, Vehicle, Car, Bike, Yacht, Powerboat, Boat
export Ford, BMW, VW, Scooter, MotorBike, Speedboat 

const KNOTS_TO_MPH = 1.151
       
struct Contact
    name::String
    email::String
    phone::String
end
                             
abstract type Vehicle end

abstract type Car <: Vehicle end
abstract type Bike <: Vehicle  end
abstract type Boat <: Vehicle end

abstract type Powerboat <: Boat end
       
struct Ford <: Car
    owner::Contact
    make::String
    fuel::String
    color::String
    engine_cc::Int64
    speed_mph::Float64
    function Ford(owner, make, engine_cc,speed_mph)
        new(owner,make,"Petrol","Black",engine_cc,speed_mph)
     end
end
       
struct BMW <: Car
    owner::Contact
    make::String
    fuel::String
    color::String
    engine_cc::Int64
    speed_mph::Float64
    function BMW(owner,make,engine_cc,speed_mph)
        new(owner,make,"Petrol","Blue",engine_cc,speed_mph)
   end
end
       
struct VW <: Car
    owner::Contact
    make::String
    fuel::String
    color::String
    engine_cc::Int64
    speed_mph::Float64
end

struct MotorBike <: Bike
    owner::Contact
    make::String
    engine_cc::Int64
    speed_mph::Float64
end
       
struct Scooter <: Bike
    owner::Contact
    make::String
    engine_cc::Int64
    speed_mph::Float64
end

mutable struct Yacht <: Boat
    owner::Contact
    make::String
    length_m::Float64
end
              
mutable struct Speedboat <: Powerboat
    owner::Contact
    make::String
    fuel::String
    engine_cc::Int64
    speed_knots::Float64
    length_m::Float64
end

function is_quicker(a::VW, b::BMW)
   if (a.speed_mph == b.speed_mph) 
      return nothing
    else
      return(a.speed_mph > b.speed_mph ? a : b)
     end
end

function is_quicker(a::Speedboat, b::Scooter)
   a_mph = KNOTS_TO_MPH * a.speed_knots
   if (a_mph == b.speed_mph) 
       return nothing
    else
       return(a_mph > b.speed_mph ? a : b)
    end
end

function is_longer(a::Yacht, b::Speedboat)
   if (a.length_m == b.length_m) 
       return nothing
    else
       return(a.length > b.length_m ? a : b)
    end
end

is_quicker(a::BMW, b::VW) = is_quicker(b,a)
is_quicker(a::Scooter, b::Speedboat) = is_quicker(b,a)
is_longer(a::Speedboat, b::Yacht) = is_longer(b,a)

end

Main.Vehicles

In [17]:
using Main.Vehicles;

malcolm = Contact("Malcolm","malcolm@abc.net","07777555999");
myCar = Ford(malcolm, "Model T", 1000, 50.0);
myBike = Scooter(malcolm, "Vespa", 125, 35.0);

james = Contact("James","james@abc.net","07777666888");
jmCar = BMW(james,"Series 500", 3200, 125.0);
jmBoat = Yacht(james,"Oceanis 44",14.6);
jmBike = MotorBike(james, "Harley", 850, 120.0);

david = Contact("David","dave@abc.net","07777222444");
dvCar = VW(david,"Golf", "diesel", "red", 1800, 85.0);
dvBoat = Speedboat(david,"Sealine","petrol", 600, 45.0, 8.2);

In [18]:
myCar.owner = david

LoadError: setfield!: immutable struct of type Ford cannot be changed

In [19]:
dvBoat.owner = malcolm

Contact("Malcolm", "malcolm@abc.net", "07777555999")

In [20]:
s = [myCar, jmCar, dvCar]
for c in s
    who = c.owner.name
    model = c.make
    println("$who has a $model car")
end

Malcolm has a Model T car
James has a Series 500 car
David has a Golf car


In [21]:
# Note that Malcolm now owns Dave's boat
#
s =[dvCar, dvBoat]  
for c in s
    who = c.owner.name
    model = c.make
    println("$who has a $model vehicle")
end

David has a Golf vehicle
Malcolm has a Sealine vehicle


In [22]:
println("James owns these vehicles: ")
s = [jmCar, jmBike, jmBoat] 
for c in s
    println("....\t$(c.make)")
end

James owns these vehicles: 
....	Series 500
....	Harley
....	Oceanis 44


In [23]:
# Make this mutable as I might change where I live
#
mutable struct Address
  name::String
  street::String
  city::String
  country::String
  postcode::String
end

In [24]:
postal = Address("Malcolm Sherrington","1 Main Street", "London", "UK", "WC2N 9ZZ");

In [25]:
const Owner = Union{Contact, Address}

Owner (alias for Union{Address, Contact})

In [26]:
struct Skiff <: Boat
   owner::Owner
   make::String
   length_m::Float64
end

In [27]:
y1 = Skiff(malcolm,"Moody 36", 11.02)

Skiff(Contact("Malcolm", "malcolm@abc.net", "07777555999"), "Moody 36", 11.02)

In [28]:
y2 = Skiff(postal,"Dufour 44", 13.47)

Skiff(Address("Malcolm Sherrington", "1 Main Street", "London", "UK", "WC2N 9ZZ"), "Dufour 44", 13.47)

In [29]:
@assert y1.length_m < y2.length_m

---

### 3D vectors

---

In [30]:
# This module uses Float64 components but could use a parameterised type {T}
#
module V3D

# import Base.+, Base.*, Base./, Base.==, Base.<, Base.>
#
import Base: +, *, /, ==, <, >, zero, one
import LinearAlgebra: norm, dot

export Vec3, norm, dist, zero, one

struct Vec3
    x::Float64
    y::Float64
    z::Float64
end

(+)(a::Vec3, b::Vec3) = Vec3(a.x+b.x, a.y+b.y, a.z+b.z)
(*)(p::Vec3, s::Real) = Vec3(p.x*s, p.y*s, p.z*s)
(*)(s::Real, p::Vec3) = p*s
(/)(p::Vec3, s::Real) = (1.0/s)*p

(==)(a::Vec3, b::Vec3) = (a.x==b.x)&&(a.y==b.y)&&(a.z==b.z) ? true : false;

dot(a::Vec3, b::Vec3) = a.x*b.x + a.y*b.y + a.z*b.z;
norm(a::Vec3) = sqrt(dot(a,a));

zero(a::Vec3) = Vec3(0.0,0.0,0.0);
one(a::Vec3)  = Vec3(1.0,1.0,1.0);

(<)(a::Vec3, b::Vec3) = norm(a) < norm(b) ? true : false;
(>)(a::Vec3, b::Vec3) = norm(a) > norm(b) ? true : false;

dist(a::Vec3, b::Vec3) = sqrt((a.x-b.x)*(a.x-b.x) + (a.y-b.y)*(a.y-b.y) + (a.z-b.z)*(a.z-b.z))

end

Main.V3D

In [31]:
using Main.V3D
using Printf

v1 = Vec3(1.2,3.4,5.6);
v2 = Vec3(2.1,4.3,6.5);
@printf "Distance between vectors is %.3f \n" dist(v1,v2)

Distance between vectors is 1.559 


In [32]:
vv = [Vec3(rand(),rand(),rand()) for i = 1:1000000];
vs = reshape(vv,1000,1000)

1000×1000 Matrix{Vec3}:
 Vec3(0.579898, 0.868496, 0.142093)     …  Vec3(0.746175, 0.139276, 0.248815)
 Vec3(0.76661, 0.266889, 0.60155)          Vec3(0.777862, 0.801402, 0.382963)
 Vec3(0.893782, 0.175078, 0.27662)         Vec3(0.236333, 0.967027, 0.486909)
 Vec3(0.454937, 0.157881, 0.440178)        Vec3(0.55442, 0.553979, 0.550037)
 Vec3(0.384037, 0.25421, 0.233728)         Vec3(0.887501, 0.00035359, 0.590963)
 Vec3(0.811274, 0.459338, 0.729067)     …  Vec3(0.866069, 0.340157, 0.196891)
 Vec3(0.425144, 0.795121, 0.336199)        Vec3(0.770606, 0.626304, 0.530395)
 Vec3(0.75144, 0.054024, 0.314219)         Vec3(0.457533, 0.389343, 0.105318)
 Vec3(0.46958, 0.182187, 0.555169)         Vec3(0.38068, 0.914203, 0.0406046)
 Vec3(0.161871, 0.365293, 0.899617)        Vec3(0.703969, 0.511545, 0.492544)
 Vec3(0.963401, 0.689999, 0.421457)     …  Vec3(0.603613, 0.780433, 0.931181)
 Vec3(0.0868113, 0.873974, 0.108336)       Vec3(0.385335, 0.280994, 0.871827)
 Vec3(0.739637, 0.679852, 0.667105)    

In [33]:
using LinearAlgebra

dv = map(v -> norm(v), vs)
dw = dv'
ee = eigen(dw)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
1000-element Vector{ComplexF64}:
 -8.697374032110387 + 0.0im
 -8.640733468987332 - 1.4481830265177589im
 -8.640733468987332 + 1.4481830265177589im
 -8.533859050177496 - 1.0093634300023806im
 -8.533859050177496 + 1.0093634300023806im
 -8.402547972095508 + 0.0im
 -8.289652253751527 - 2.292319795167901im
 -8.289652253751527 + 2.292319795167901im
 -8.100539052020537 - 1.1559491382236424im
 -8.100539052020537 + 1.1559491382236424im
 -8.080353836336261 - 1.5348982917964933im
 -8.080353836336261 + 1.5348982917964933im
 -8.056490204070359 - 2.155995758071399im
                    ⋮
  8.153226056183055 + 1.5211417962377771im
  8.219720194159432 - 0.9115382985621477im
  8.219720194159432 + 0.9115382985621477im
  8.391566698016284 - 2.5510552235374355im
  8.391566698016284 + 2.5510552235374355im
  8.427399087615786 - 0.5103192661508726im
  8.427399087615786 + 0.5103192661508726im
  8.464320153718987 - 1.2093374060207094

In [34]:
k = 0
for i in 1:length(vv)
  if norm(vv[i]) < 1.0 
    global k +=1 
  end
end
@printf "Estimate of PI is %9.5f\n" 6.0*k/length(vv)

Estimate of PI is   3.14558


In [35]:
#
# Defining the Vec3 parameterically and the rest of the module is unchanged
# i.e.
#
# immutable Vec3P{T}
#    x::T
#    y::T
#    z::T
# end
#
module V3P
#
# import Base.+, Base.*, Base./, Base.norm, Base.==, Base.<, Base.>
#
import Base: +, *, /, ==, <, >
import LinearAlgebra: norm, dot
export Vec3P, norm, distp

struct Vec3P{T}
    x::T
    y::T
    z::T
end

(+)(a::Vec3P, b::Vec3P) = Vec3P(a.x+b.x, a.y+b.y, a.z+b.z)
(*)(p::Vec3P, s::Real) = Vec3P(p.x*s, p.y*s, p.z*s)
(*)(s::Real, p::Vec3P) = p*s
(/)(p::Vec3P, s::Real) = (1.0/s)*p

(==)(a::Vec3P, b::Vec3P) = (a.x==b.x)&&(a.y==b.y)&&(a.z==b.z) ? true : false;

dot(a::Vec3P, b::Vec3P) = a.x*b.x + a.y*b.y + a.z*b.z;
norm(a::Vec3P) = sqrt(dot(a,a));

(<)(a::Vec3P, b::Vec3P) = norm(a) < norm(b) ? true : false;
(>)(a::Vec3P, b::Vec3P) = norm(a) > norm(b) ? true : false;

distp(a::Vec3P, b::Vec3P) = sqrt((a.x-b.x)*(a.x-b.x) + (a.y-b.y)*(a.y-b.y) + (a.z-b.z)*(a.z-b.z))

end

Main.V3P

In [36]:
using Main.V3P

z1 = Vec3P{Complex}(1+2im,2+3im,3+4im);
z2 = Vec3P{Complex}(3-2im,4-3im,5-4im);
z1 + z2

Vec3P{Complex{Int64}}(4 + 0im, 6 + 0im, 8 + 0im)

In [37]:
zn = norm(z1 + z2)        # => sqrt(116) but IS a complex number.
@assert zn == sqrt(116)

# Notice that the vaule is returned as a complex.
zn

10.770329614269007 + 0.0im

In [38]:
r1 = Vec3P{Rational}(11//7,13//5,8//17)
r2 = Vec3P{Rational}(17//9,23//15,28//17)
r1 + r2

Vec3P{Rational{Int64}}(218//63, 62//15, 36//17)

In [39]:
# Norm is promoted to a Real as it can not be represented as a Rational
#
norm(r1 + r2)

5.791603442602598

---

### It is possible to extend the above to an *nD* vector simply using static arrays.

---

In [40]:
module VNX
using StaticArrays

import Base: +, *, /, ==, <, >
import LinearAlgebra: norm, dot
export VecN, norm, distx

struct VecN
    sv::SVector;
end

sizeof(a::VecN) = length(a.sv)
sOK(a::VecN, b::VecN) = 
 (sizeof(a) == sizeof(b)) ? true : throw(BoundsError("Vector of different lengths"));

(+)(a::VecN, b::VecN) = [a.sv[i] + b.sv[i] for i in 1:sizeof(a) if sOK(a,b)]
(*)(x::Real, a::VecN) = [a.sv[i]*x for i in 1:sizeof(a)]
(*)(a::VecN, x::Real) = x*a
(/)(a::VecN, x::Real) = [a.sv[i]/x for i in 1:sizeof(a)]

(==)(a::VecN, b::VecN) = any([(a.sv[i] == b.sv[i]) for i in 1:sizeof(a) if sOK(a,b)])
                            
dot(a::VecN, b::VecN) = sum([a.sv[i]*b.sv[i] for i in 1:sizeof(a) if sOK(a,b)])
norm(a::VecN) = sqrt(dot(a,a));

(<)(a::VecN, b::VecN) = norm(a) < norm(b) ? true : false;
(>)(a::VecN, b::VecN) = norm(a) > norm(b) ? true : false;                                            
                                            
distx(a::VecN, b::VecN) = sum(map(x -> x*x,[a.sv[i]-b.sv[i] for i in 1:sizeof(a) if sOK(a,b)]))

end

Main.VNX

In [41]:
# Generate K 4-vectors
#
using Main.VNX, StaticArrays

K = 10^5; 
vv = Array{VecN}(undef,K);

for j = 1:K
  vv[j] =  VecN(@SVector [rand() for i = 1:4])
end

In [42]:
# Sum up the vectors which lie within the 4-ball
#
s = 0;
for j = 1:K 
  if (norm(vv[j]) < 1.0) s += 1 end
end

In [43]:
# Volume of the unit 4-ball is 2*π*π
# The count is 1/16th of the volume
# 
mypi = sqrt(32*s/K)

3.127529376360836